In [1]:
%load_ext line_profiler
import numba
import numpy as np
import scipy as sp
%load_ext watermark
%watermark

2017-11-23T17:38:43+01:00

CPython 3.6.2
IPython 6.1.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.4.92-18.36-default
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [2]:
import atm_c14_data
import c14_individuals_data

In [3]:
# np.random.seed(42)
# data = c14_individuals_data.c14_individual_data.head().copy()
# data['Dbirth'] = np.random.randint(0, 5, len(data))
# data['Dcoll'] = np.random.randint(6, 10, len(data))
# data['d14C'] = C_atm(data['Dcoll']) - 0.5
# data

<function atm_c14_data.interpolate.<locals>.<lambda>>

In [50]:
C_atm = atm_c14_data.Klag

def f(C, t):
    return 0.5 * (C + C_atm(t))

# def find_nearest_value(array,value):
#     return (np.abs(array-value)).argmin()

# find_nearest = np.vectorize(find_nearest_value, excluded=[0])

def find_nearest(CC,C):
    dC = (CC[1] - CC[0])/ (CC[-1] - CC[0]) 
    i = np.rint((C - CC[0])/ (CC[-1] - CC[0]) /dC)
    i = i.astype(int)
    return i

def k(t, C, CC):
    return find_nearest(CC, f(C, t))

def kCC(t, CC):
    return k(t, CC, CC)

@numba.jit
def target_matrix(t, CC):
    T = np.zeros(2*[len(CC)])
    k = kCC(t, CC)
    for j, i in enumerate(k):
        T[i, j] = 1
    return T

def rate_matrix(t, CC):
    T = target_matrix(t, CC)
    A = T - np.eye(len(CC))
    return A

# def C(Dbirth, Dcoll, r, n=3, dt = 0.1):
#     C_min = C_atm(Dbirth)
#     C_max = C_atm(Dcoll)
    
#     CC = np.linspace(C_min, C_max, n)
    
#     N0 = np.zeros(n)
#     N0[find_nearest(CC, C_atm(0))] = 1
    
#     t = Dbirth
#     N = N0
#     while t<Dcoll:
#         A = rate_matrix(t, CC)
#         N += r * dt * A @ N
#         t += dt
    
#     C = np.average(CC, weights=N)
    
#     return C

def C_L1(Dbirth, Dcoll, r, n=3, dt = 0.1):
    C_min = atm_c14_data.c14_data['d14C'].min()
    C_max = atm_c14_data.c14_data['d14C'].max()
    
    CC = np.linspace(C_min, C_max, n)
    
    N0 = np.zeros(n)
    N0[find_nearest(CC, C_atm(0))] = 1
    
    def rhs(N, t0):
        A = rate_matrix(t0, CC)
        dNdt = r * A @ N
        return dNdt
    
    sol = sp.integrate.odeint(rhs, N0, (Dbirth, Dcoll), mxstep=int(1e5))
    N = sol[-1]
    C = np.average(CC, weights=N)
    return C

In [49]:
C(1930, 1960, 10.0, n=100)

0.16665362520789087

In [38]:
C_atm(1960)

0.18409999999999999

In [14]:
n = 100
dt = 0.005
print(C(0, 8, 1, n=n, dt=dt))
%timeit C(0, 8, 1, n=n, dt=dt)
print(C2(0, 8, 1, n=n, dt=dt))
%timeit C2(0, 8, 1, n=n, dt=dt)

6.04104004877
81.5 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.03634731599
418 ms ± 6.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Tuning calculation of the target matrix

In [192]:
print(C(0, 8, 1, n=10, dt=0.01))
%lprun -f find_nearest C(0, 8, 1, n=10, dt=0.01)
%timeit C(0, 8, 1, n=10, dt=0.01)

6.01784828878
21.1 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


New find_nearest

In [102]:
%timeit C(0, 8, 1, n=10, dt=0.01)

26.9 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Before

In [19]:
%timeit C(0, 8, 1, n=100, dt=0.01)

690 ms ± 58.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
